# **Taller 1**
### **Integrantes:**
* Diego Felipe Carvajal Lombo (201911910)
* Brenda Catalina Barahona Pinilla (201812721)
* Sergio Julian Zona Moreno (201914936)

In [5]:
# Posible instalación necesaria
#!conda install -c conda-forge scikit-surprise

# **Carga y muestreo de los datos**

In [6]:
# Importación de librerias
seed = 161
import pandas as pd
import numpy as np
import hashlib

# Se importa la librería de tiempo para medir cuánto se demora en encontrar los hiperparámetros con cada modelo.
import time
import math

# Librerias CUDA
import cudf
#cudf.set_option("spill", True)

# Database
import sqlite3

# Gráficos
import matplotlib.pyplot as plt

# Importamos la librería del SR
import os
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy

#Para garantizar reproducibilidad en resultados
import random
seed = 2023
#random.seed(seed)
#np.random.seed(seed)

# Importar/Exportar modelos
from joblib import dump, load

In [7]:
!nvidia-smi

Mon Mar  6 23:27:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 L...    On | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P8               13W /  N/A|    168MiB /  6144MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Carga de datos**

In [8]:
# Se cargan el conjunto total de los datos. 
# Leer TSV: https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
# Tokenizing data error: https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data

fields = ["userid", "timestamp", "musicbrainz-artist-id", "artist-name", "musicbrainz-track-id", "track-name"]

df_data=pd.read_csv('../data/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t',
                     on_bad_lines='skip', skipinitialspace=True, names=fields)

In [9]:
df_data = cudf.from_pandas(df_data)

In [10]:
# Cantidad de datos y número de variables
df_data.shape

(19098853, 6)

In [11]:
# Ejemplo de muestra de los datos.
df_data.tail(5)

,userid,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
19098848,user_001000,2008-01-27T22:02:35Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,a490cabc-1e5c-4807-86c7-740c31a50009,Please Be Patient With Me
19098849,user_001000,2008-01-27T21:56:52Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,3e92e447-9e1f-440d-bc00-6734469880c5,Shake It Off
19098850,user_001000,2008-01-27T21:52:36Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,93d044e6-1bbb-46a6-ac8e-283382a89e6f,Side With The Seeds
19098851,user_001000,2008-01-27T21:49:12Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,5ac4386f-6146-4389-a762-4b43f362d2c8,Sky Blue Sky
19098852,user_001000,2008-01-27T21:43:14Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,3acc99bc-a349-420f-ad28-7095eb3533c9,Impossible Germany


In [12]:
# Tipos de las variables al cargar, todas son objetos.
df_data.dtypes

userid                   object
timestamp                object
musicbrainz-artist-id    object
artist-name              object
musicbrainz-track-id     object
track-name               object
dtype: object

In [13]:
# Número de valores nulos en filas.
df_plot = df_data.isnull().sum().sort_values()
df_plot

userid                         0
timestamp                      0
artist-name                    0
track-name                    12
musicbrainz-artist-id     600848
musicbrainz-track-id     2162719
dtype: int64

In [14]:
# Existen múltiples filas con valores nulos. Esto se debe a que son llaves foráneas.
#plt.barh(df_plot.index, df_plot.values)

# **Perfilamiento y entendimiento de los datos**
Obtendremos estadísticas descriptivas pertinentes y posteriormente ingresaremos el conjunto de datos a Pandas Profiling para obtener un reporte adecuado de correlación e interacción entre variables.

In [15]:
# Obtención de estadísticas descriptivas.
df_data.describe()

,userid,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
count,19098853,19098853,18498005,19098853,16936134,19098841
unique,992,17454730,107296,173921,960403,1083472
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,183103,248,115099,115099,3991,17561


In [16]:
# Obtención de estadísticas descriptivas.
df_data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 19098853 entries, 0 to 19098852
Data columns (total 6 columns):
 #   Column                 Dtype
---  ------                 -----
 0   userid                 object
 1   timestamp              object
 2   musicbrainz-artist-id  object
 3   artist-name            object
 4   musicbrainz-track-id   object
 5   track-name             object
dtypes: object(6)
memory usage: 2.7+ GB


In [24]:
# Contamos el número de veces que una persona escuchó una canción. Y tomamos esto como matriz de utilidad.
# La segunda línea quita el multi-index.
df_user_track = df_data.groupby(['userid', 'track-name']).count().sort_values('timestamp', ascending=False)['timestamp'].to_frame()
df_user_track = df_user_track.reset_index(level=[0,1])

# Optimizamos la memoria.
# Link: https://towardsdatascience.com/memory-efficient-data-science-types-53423d48ba1d
df_user_track['timestamp'] = df_user_track['timestamp'].astype(np.uint16)
df_user_track.rename(columns={"userid":"user_id","track-name":"track_name","timestamp": "rating"}, inplace=True)
df_user_track

,user_id,track_name,rating
0,user_000008,Heartless,2119
1,user_000008,See You In My Nightmares,2069
2,user_000008,Say You Will,2065
3,user_000008,Welcome To Heartbreak (Feat. Kid Cudi),2059
4,user_000008,Love Lockdown,2059
...,...,...,...
4407905,user_000298,Each Other,1
4407906,user_000666,Walking Thru The Park,1
4407907,user_000783,Your Mistake / Victim In Pain (Live),1
4407908,user_000382,Junesong Provision,1


In [25]:
#df_test = df_user_track.groupby('user_id').count()
df_test = df_user_track.loc[df_user_track['user_id'] == 'user_000008']

df_test['rating'].iloc[-1]
df_test

,user_id,track_name,rating
0,user_000008,Heartless,2119
1,user_000008,See You In My Nightmares,2069
2,user_000008,Say You Will,2065
3,user_000008,Welcome To Heartbreak (Feat. Kid Cudi),2059
4,user_000008,Love Lockdown,2059
...,...,...,...
4348908,user_000008,Fair To You,1
4354128,user_000008,Trans Boulogne Express,1
4358356,user_000008,Had Enough,1
4362499,user_000008,The Phone,1


In [26]:
from cuml.preprocessing import MinMaxScaler
#from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1, 5))
df_test = df_test.reset_index() # Problemas con los índices al transformar en Sklearn.
df_test['rating'] = scaler.fit_transform(df_test[['rating']])
df_test = df_test.set_index('index')
df_test = df_test.reset_index(level=[0,1])
df_test

,index,user_id,track_name,rating
0,0,user_000008,Heartless,5.000000
1,1,user_000008,See You In My Nightmares,4.905571
2,2,user_000008,Say You Will,4.898017
3,3,user_000008,Welcome To Heartbreak (Feat. Kid Cudi),4.886686
4,4,user_000008,Love Lockdown,4.886686
...,...,...,...,...
603,4348908,user_000008,Fair To You,1.000000
604,4354128,user_000008,Trans Boulogne Express,1.000000
605,4358356,user_000008,Had Enough,1.000000
606,4362499,user_000008,The Phone,1.000000


In [22]:
df_test['rating']

0      5.000000
1      4.905571
2      4.898017
3      4.886686
4      4.886686
         ...   
603    1.000000
604    1.000000
605    1.000000
606    1.000000
607    1.000000
Name: rating, Length: 608, dtype: float64

In [27]:
# Contamos el número de veces que una persona escuchó un artista. Y tomamos esto como matriz de utilidad.
# La segunda línea quita el multi-index.
df_user_artist = df_data.groupby(['userid', 'artist-name']).count().sort_values('timestamp', ascending=False)['timestamp'].to_frame()
df_user_artist = df_user_artist.reset_index(level=[0,1])

# Optimizamos la memoria.
# Link: https://towardsdatascience.com/memory-efficient-data-science-types-53423d48ba1d
df_user_artist['timestamp'] = df_user_artist['timestamp'].astype(np.uint16)
df_user_artist.rename(columns={"userid":"user_id","artist-name":"artist_name","timestamp": "rating"}, inplace=True)
df_user_artist

,user_id,artist_name,rating
0,user_000008,Kanye West,26496
1,user_000141,Chemistry,25609
2,user_000499,The Knife,18597
3,user_000889,Soilwork,15566
4,user_000084,Britney Spears,14614
...,...,...,...
897414,user_000966,Mcg'Z,1
897415,user_000758,Eric Burdon And The Animals,1
897416,user_000222,Fear Factory,1
897417,user_000412,Minotaur Shock,1


In [ ]:
# Analizamos el consumo en memoria de los DF's
df_user_track.info()
print("------------------------------------------")
df_user_artist.info()

In [ ]:
# Exportamos los DF's
df_user_track.index.name='id'
df_user_track.to_csv("../data/processed/user_track.csv")
df_user_artist.index.name='id'
df_user_artist.to_csv("../data/processed/user_artist.csv")

In [ ]:
# Este código genera la matriz de 1's y 0s. Realiza un pivote y cuenta las coincidencias.
#df_user_track = df_user_track.pivot(index='userid', columns='artist-name', values='timestamp')

In [ ]:
# Se cargan el conjunto total de los datos. 
# Leer TSV: https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
# Tokenizing data error: https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data

df_users=pd.read_csv('../data/userid-profile.tsv', sep='\t')
df_users['registered'] = pd.to_datetime(df_users['registered']).apply(str)
df_users.rename(columns={"#id":"user_id"}, inplace=True)
df_users.info()

In [ ]:
# Creamos la columna que tendrá el Password. Para facilitar nuestra simulación, esta columna será: hash256('user_id'+'123.')
def hash(x):
    h = hashlib.new('sha256')
    h.update(x.encode())
    return h.hexdigest()

df_users['password_hash'] = df_users['user_id'].apply(lambda x: hash(str(x)+'123.'))
df_users

In [ ]:
df_users.to_csv("../data/processed/users.csv")

In [ ]:
# Se genera un reporte de analítica. Demora menos de 1 minuto aproximadamente.
#profile = ProfileReport(df_user_artist.to_pandas(), title="Pandas Profiling Report", minimal=True)
#profile.to_file("reporte.html")

# **Creación de la base de datos SQLite3**
Creamos una base de datos SQLite3, con base en los DF's generados.

In [ ]:
# En caso de que no se pueda utilizar CUDF.
df_user_track = df_user_track.to_pandas()
df_user_artist = df_user_artist.to_pandas()

In [ ]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../backend/data/data.db')

# Create a cursor object
cur = conn.cursor()

# Create a new table in the database
cur.execute('CREATE TABLE IF NOT EXISTS user_track (id INTEGER PRIMARY KEY, user_id TEXT, track_name TEXT, rating INTEGER)')

for index, row in df_user_track.iterrows():
    cur.execute('INSERT INTO user_track (id, user_id, track_name, rating) VALUES (?, ?, ?, ?)', 
                (index, row['user_id'], row['track_name'], row['rating']))

# Commit the changes and close the database connection
conn.commit()
cur.close()
conn.close()

In [ ]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../backend/data/data.db')

# Create a cursor object
cur = conn.cursor()

# Create a new table in the database
cur.execute('CREATE TABLE IF NOT EXISTS user_artist (id INTEGER PRIMARY KEY, user_id TEXT, artist_name TEXT, rating INTEGER)')

for index, row in df_user_artist.iterrows():
    cur.execute('INSERT INTO user_artist (id, user_id, artist_name, rating) VALUES (?, ?, ?, ?)', 
                (index, row['user_id'], row['artist_name'], row['rating']))

# Commit the changes and close the database connection
conn.commit()
cur.close()
conn.close()

In [ ]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../backend/data/data.db')

# Create a cursor object
cur = conn.cursor()

# Create a new table in the database
cur.execute('CREATE TABLE IF NOT EXISTS user (user_id PRIMARY KEY, gender TEXT, age INTEGER, country TEXT, registered TIMESTAMP, password_hash TEXT)')

for index, row in df_users.iterrows():
    cur.execute('INSERT INTO user (user_id, gender, age, country, registered, password_hash) VALUES (?, ?, ?, ?, ?, ?)', 
                (row['user_id'] , row['gender'], row['age'], row['country'], row['registered'], row['password_hash']))

# Commit the changes and close the database connection
conn.commit()
cur.close()
conn.close()

# **Creación de modelo de filtrado colaborativo basado en similitud con usuarios o items cercanos**

Surprise cuenta con la implementación de los modelos colaborativos dentro de la clase [KNNBasic](https://surprise.readthedocs.io/en/stable/knn_inspired.html) 

El modelo recibe los siguientes parámetros: 


*   k: El máximo número de vecinos con el que se hará la extrapolación
*   min_k : El mínimo número de vecinos con el que se extrapolará un rating
*   sim_options : Opciones de similitud pasadas como un diccionario de python, aqui se le configura al modelo el tipo de similitud a usar para encontrar los vecinos y si la extrapolación debe hacerse usando usuarios o items similares. Revise el formato y similitudes disponibles en surprise en [este link](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measure-configuration)





In [ ]:
reader = Reader( rating_scale = ( 1, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'user_id', 'item_id', 'rating' ] ], reader )

train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)

In [27]:
# se crea un modelo knnbasic item-item con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)

In [45]:
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Una vez cargados los ratings al modelo, se puede realizar una predicción para un usuario, en este caso vamos a calcular la predicción que el modelo esta realizando para la primera entrada del dataset de test la predicción para el usuario con id 154 y el item con id 302, que corresponde a la película L.A. Confidential (1997)

---



In [47]:
#Verifique la propiedad est de la predicción
algo.predict(154,302)

Prediction(uid=154, iid=302, r_ui=None, est=4.249202065377386, details={'actual_k': 20, 'was_impossible': False})

In [ ]:
items[items['movie id']==302]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
301,302,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


Como podemos ver, la predicción (4.24) del modelo no esta alejada de lo que realmente opinó el usuario  (4.0)

Para medir la calidad de la predicción para todos los usuarios e items del dataset de prueba, vamos a comparar lo que dice el modelo de predicción vs lo que dice el conjunto de prueba, para esto vamos a usar la métrica [RMSE](https://surprise.readthedocs.io/en/stable/accuracy.html#surprise.accuracy.rmse)

Inicialmente calculamos la predicción para todos los elementos del conjunto de test

In [48]:
test_predictions=algo.test(test_set)

In [49]:
#5 primeras predicciones
test_predictions[0:5]

[Prediction(uid=154, iid=302, r_ui=4.0, est=4.249202065377386, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=896, iid=484, r_ui=4.0, est=2.8976616922320964, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=230, iid=371, r_ui=4.0, est=4.1968378983432535, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=234, iid=294, r_ui=3.0, est=3.0978778071364186, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=25, iid=729, r_ui=4.0, est=4.148885433441103, details={'actual_k': 20, 'was_impossible': False})]

Ahora se mide el RMSE de las predicciones vs el valor del dataset

In [50]:
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

RMSE: 1.0453


1.0453343976192102

##Utilice las siguientes celdas para encontrar la respuesta a las siguientes preguntas

¿Cuál es el RMSE de un modelo usuario-usuario con los mismos parámetros de similitud?

¿Cuál es el efecto de cambiar el número de vecinos en la calidad del modelo usuario-usuario ?

In [51]:
# se crea un modelo knnbasic user-user con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud user-user
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)

In [52]:
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [53]:
test_predictions=algo.test(test_set)

In [65]:
#5 primeras predicciones
test_predictions[0:5]

[Prediction(uid=154, iid=302, r_ui=4.0, est=4.35, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=896, iid=484, r_ui=4.0, est=4.350586293776335, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=230, iid=371, r_ui=4.0, est=3.607980194649977, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=234, iid=294, r_ui=3.0, est=3.6497107625654204, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=25, iid=729, r_ui=4.0, est=3.8002893050257525, details={'actual_k': 20, 'was_impossible': False})]

In [66]:
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

RMSE: 1.0166


1.0166414414593592

In [67]:
# se crea un modelo knnbasic user-user con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud user-user
               }
algo = KNNBasic(k=10, min_k=2, sim_options=sim_options)
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)
test_predictions=algo.test(test_set)
#5 primeras predicciones
test_predictions[0:5]
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0386


1.0385724992200804

In [68]:
# se crea un modelo knnbasic user-user con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud user-user
               }
algo = KNNBasic(k=5, min_k=2, sim_options=sim_options)
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)
test_predictions=algo.test(test_set)
#5 primeras predicciones
test_predictions[0:5]
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0839


1.0838620997926653

In [69]:
# se crea un modelo knnbasic user-user con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud user-user
               }
algo = KNNBasic(k=35, min_k=2, sim_options=sim_options)
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)
test_predictions=algo.test(test_set)
#5 primeras predicciones
test_predictions[0:5]
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0101


1.0100769609066584

##Generando listas de predicciones para los usuarios

Retomemos nuestro modelo inicial y ajustémolo con todos los ratings disponibles

Para generar una lista de recomendación se debe crear un dataset de "test" con las entradas faltantes de la matriz utilidad para que el modelo cree las predicciones (terminar de llenar la matriz de utilidad)



In [70]:
surprise_dataset.build_full_trainset()

In [71]:
#Se crea el dataset para modelo 
rating_data=surprise_dataset.build_full_trainset()
# Se crea dataset de "prueba" con las entradas faltantes para generar las predicciones
test=rating_data.build_anti_testset()

# se crea el mismo modelo que el del ejemplo
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)
algo.fit(rating_data)
predictions=algo.test(test)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [72]:
#10 primeras predicciones
predictions[0:10]

[Prediction(uid=196, iid=302, r_ui=3.52986, est=3.4998074068929244, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=377, r_ui=3.52986, est=3.531852919263047, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=51, r_ui=3.52986, est=3.59549677885145, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=346, r_ui=3.52986, est=3.5471070543165877, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=474, r_ui=3.52986, est=3.7990088769026116, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=265, r_ui=3.52986, est=3.4464297466659812, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=465, r_ui=3.52986, est=3.5022156502733557, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=451, r_ui=3.52986, est=3.6948301041629965, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=86, r_ui=3.52986, e

In [73]:
#Predicciones para usuario 196
user_predictions=list(filter(lambda x: x[0]==196,predictions))

In [74]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [75]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [76]:
user_predictions

[Prediction(uid=196, iid=1309, r_ui=3.52986, est=4.5, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=196, iid=1310, r_ui=3.52986, est=4.5, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=196, iid=1676, r_ui=3.52986, est=4.25, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=196, iid=1675, r_ui=3.52986, est=4.25, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=196, iid=1289, r_ui=3.52986, est=4.202435603862276, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=1643, r_ui=3.52986, est=4.097225899300029, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=1593, r_ui=3.52986, est=4.090909090909091, details={'actual_k': 11, 'was_impossible': False}),
 Prediction(uid=196, iid=935, r_ui=3.52986, est=4.05136705364539, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=196, iid=1216, r_ui=3.52986, est=4.0506920732628675, details={'actual_k': 20, 'was_i

In [77]:
#Se convierte a dataframe
labels = ['movie id', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

In [78]:
#Lo unimos con el dataframe de películas
df_predictions.merge(items[['movie id','movie title','IMDb URL ']], how='left', on='movie id')

,movie id,estimation,movie title,IMDb URL
0,1309,4.500000,"Very Natural Thing, A (1974)",http://us.imdb.com/M/title-exact?Very%20Natura...
1,1310,4.500000,"Walk in the Sun, A (1945)",http://us.imdb.com/M/title-exact?Walk%20in%20t...
2,1676,4.250000,"War at Home, The (1996)",http://us.imdb.com/M/title-exact?War%20at%20Ho...
3,1675,4.250000,"Sunchaser, The (1996)","http://us.imdb.com/M/title-exact?Sunchaser,%20..."
4,1289,4.202436,Jack and Sarah (1995),http://us.imdb.com/M/title-exact?Jack%20and%20...
5,1643,4.097226,Angel Baby (1995),http://us.imdb.com/Title?Angel+Baby+(1995/I)
6,1593,4.090909,Death in Brunswick (1991),http://us.imdb.com/M/title-exact?Death%20in%20...
7,935,4.051367,Paradise Road (1997),http://us.imdb.com/M/title-exact?Paradise%20Ro...
8,1216,4.050692,Kissed (1996),http://us.imdb.com/M/title-exact?Kissed%20%281...
9,1312,4.049247,"Pompatus of Love, The (1996)",http://us.imdb.com/M/title-exact?Pompatus%20of...


# **Modelo de recomendación**
Generamos dos modelos de recomendación, uno item-item y otro user-user

In [ ]:
# EXPORTAMOS EL MODELO: 
# Usamos la libreria joblib.
filename = 'logistic_model.joblib'
# Se guarda
dump(final_model, filename) 